# Training a model in TF

In [1]:
import numpy as np
import tensorflow as tf

In [2]:
from tensorflow.contrib.learn.python.learn import datasets
dataset = datasets.load_iris()

# split into input data (x) and GT labels (y)
data_x = dataset.data
data_y = dataset.target
print data_x.shape
print data_y.shape

(150, 4)
(150,)


Splitting the data into testing and training data. We shuffle the data first, as the dataset is ordered in terms of labels.

In [3]:
new_indices = np.random.permutation(data_y.shape[0])

train_x = data_x[new_indices[:125],:]
train_y = data_y[new_indices[:125]]

test_x = data_x[new_indices[125:],:]
test_y = data_y[new_indices[125:]]

## Step 1: Building the computation graph

In [4]:
# placeholder for data x (4 attributes), one prediction label
x    = tf.placeholder("float", shape=[None, 4])
y_GT = tf.placeholder("int64", shape=[None, ])

# model parameters
n_hidden = 100
W_h = tf.Variable(0.1*tf.random_normal([4, n_hidden]), name="W_h")
b_h = tf.Variable(tf.random_normal([n_hidden]), name="b_h")
hidden_layer = tf.matmul(x, W_h) + b_h

# model parameters
W = tf.Variable(0.1*tf.random_normal([n_hidden, 3]), name="W")
b = tf.Variable(tf.zeros([3]), name="b")

# putting the model together
y = tf.nn.softmax(tf.matmul(hidden_layer, W) + b)

In [5]:
print y

Tensor("Softmax:0", shape=(?, 3), dtype=float32)


## Step 2: Specify loss and training/optimization

In [6]:
# cross-entropy loss
#cross_entropy = -tf.reduce_sum(tf.one_hot(y_GT, depth=3) * tf.log(y+1e-10))
cross_entropy = tf.reduce_sum(tf.nn.softmax_cross_entropy_with_logits(labels=tf.one_hot(y_GT, depth=3), logits=(tf.matmul(hidden_layer, W) + b)))
# define optimizer
opt = tf.train.GradientDescentOptimizer(0.001)
train_op = opt.minimize(cross_entropy)

In [7]:
cross_entropy

<tf.Tensor 'Sum:0' shape=() dtype=float32>

## Step 3: Launch a session

In [8]:
# create operation to inialize all variables
init_op = tf.initialize_all_variables()

In [9]:
# launch session
with tf.Session() as sess:
    sess.run(init_op)
    
    print W
    #print sess.run(W) #print W.eval()

In [10]:
# one step of training
with tf.Session() as sess:
    
    sess.run(init_op)
    sess.run(train_op, feed_dict={x: data_x, y_GT: data_y})
    
    xen = sess.run(cross_entropy, feed_dict={x: data_x, y_GT: data_y})
    print xen
    

249.704


### Adding summary variables that want to track during training

We will track the cross-entropy, and accuracy. The log files will be saves in a separate folder **\tf_logs**.

To visualize the plots/learning curves, go to your terminal and execute:

```
>> tensorboard --logdir="\tf_logs"
```

And then open your browser(preferably Chrome) and navigate to: http://127.0.1.1:6006.


In [11]:
# define the accuracy 
accuracy = tf.reduce_mean(tf.cast(tf.equal(y_GT, tf.argmax(y, 1)), tf.float32))

In [12]:
# summary 
tf.scalar_summary("cross_entropy", cross_entropy)
tf.scalar_summary("accuracy", accuracy)
summary_op = tf.merge_all_summaries()

In [13]:
# multiple steps of training

with tf.Session() as sess:
    sess.run(init_op)
    
    summary_writer  = tf.train.SummaryWriter('tf_logs', sess.graph)
    for iterIndex in range(200):
        sess.run(train_op, feed_dict={x: train_x, y_GT: train_y})
        
        summary = sess.run(summary_op, feed_dict={x: train_x, y_GT: train_y})
        summary_writer.add_summary(summary, iterIndex)
        
        if iterIndex%10==0:
            
            xen = sess.run(cross_entropy, feed_dict={x: test_x, y_GT: test_y})
            print("Iter %d -- Cross-entropy: %f"%(iterIndex,xen))
            
            acc = sess.run(accuracy, feed_dict={x: test_x, y_GT: test_y})
            print("Iter %d -- Accuracy: %f"%(iterIndex,acc))

Iter 0 -- Cross-entropy: 140.537842
Iter 0 -- Accuracy: 0.400000
Iter 10 -- Cross-entropy: 12.670593
Iter 10 -- Accuracy: 0.960000
Iter 20 -- Cross-entropy: 9.923141
Iter 20 -- Accuracy: 0.920000
Iter 30 -- Cross-entropy: 8.319332
Iter 30 -- Accuracy: 0.960000
Iter 40 -- Cross-entropy: 7.398432
Iter 40 -- Accuracy: 0.920000
Iter 50 -- Cross-entropy: 11.380806
Iter 50 -- Accuracy: 0.680000
Iter 60 -- Cross-entropy: 8.780022
Iter 60 -- Accuracy: 0.800000
Iter 70 -- Cross-entropy: 9.253835
Iter 70 -- Accuracy: 0.800000
Iter 80 -- Cross-entropy: 8.677464
Iter 80 -- Accuracy: 0.800000
Iter 90 -- Cross-entropy: 8.440955
Iter 90 -- Accuracy: 0.800000
Iter 100 -- Cross-entropy: 8.134848
Iter 100 -- Accuracy: 0.800000
Iter 110 -- Cross-entropy: 7.754847
Iter 110 -- Accuracy: 0.800000
Iter 120 -- Cross-entropy: 7.417487
Iter 120 -- Accuracy: 0.800000
Iter 130 -- Cross-entropy: 7.047817
Iter 130 -- Accuracy: 0.800000
Iter 140 -- Cross-entropy: 6.641553
Iter 140 -- Accuracy: 0.880000
Iter 150 -- C